# TP : Génération augmentée de récupération (RAG)

## Installation et importation des bibliothèques

In [ ]:
!pip install -U langchain-community  # Bibliothèque facilitant l'intégration de LLM dans des applications
!pip install faiss-gpu  # FAISS est une méthode d'indexation et de récupération à base d'embeddings
!pip install datasets  # Téléchargement de jeux de données

# Ligne à supprimer une fois que transformers 4.46.4 sera sorti
!pip install -U git+https://github.com/huggingface/transformers  # Mise à jour de transformers, nous avons besoin de la version la plus récente

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-22dm4u83
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-22dm4u83
  Resolved https://github.com/huggingface/transformers to commit 5523e38b553ff6c46b04d2376870fcd842feeecc
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from transformers import AutoTokenizer
from transformers.agents import Tool

from langchain_community.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from transformers.agents import HfApiEngine, Agent, ReactJsonAgent

import logging
import datasets
import pandas as pd
import numpy as np

from datasets import load_dataset
from tqdm import tqdm  # Barres de chargement

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


## Collection de documents

La première étape consiste à créer une collection de documents qui servira de base de données externe à notre modèle. Pour cela, nous utiliserons le jeu de données [Cities_Wikipedia_Information](https://huggingface.co/datasets/Sketched33/Cities_Wikipedia_Information/), qui contient sous forme de texte le contenu des articles Wikipédia à propos des plus grandes villes du monde.

Commençons par observer ce jeu de données :

In [ ]:
ds = load_dataset("Sketched33/Cities_Wikipedia_Information", split="train")
df = pd.DataFrame(ds)
df

,city_name,wikipedia_content,latitude,longitude
0,Tokyo,"Tokyo (; Japanese: 東京, Tōkyō, [toːkʲoː] ), off...",35.6897,139.6922
1,Jakarta,Jakarta (; Indonesian pronunciation: [dʒaˈkart...,-6.1750,106.8275
2,Delhi,"Delhi, officially the National Capital Territo...",28.6100,77.2300
3,Guangzhou,Guangzhou is the capital and largest city of G...,23.1300,113.2600
4,Mumbai,"Mumbai ( , Marathi: [ˈmumbəi], IAST: Muṃbaī; f...",19.0761,72.8775
...,...,...,...,...
7814,Godinlabe,"Godinlabe (Somali: Godinlaba, is a town in Gal...",5.8794,46.6308
7815,Sechura,"Sechura is a town in northwestern Peru, 50 km ...",-5.5576,-80.8223
7816,Al Wajh,"Al Wajh (Arabic: الوجه), also written Al Wejh,...",26.2833,36.4167
7817,Abu Hamad,"Abu Hamad (Arabic: أبو حمد, Sudanese Arabic [a...",19.5433,33.3378


In [ ]:
# Longueur des articles en caractères
lengths = df['wikipedia_content'].str.len()
print(lengths.sum())
lengths.describe()

110059340


,wikipedia_content
count,7819.000000
mean,14075.884384
std,18219.283805
min,68.000000
25%,2769.500000
50%,6874.000000
75%,17343.500000
max,228327.000000


Comme on le voit, les longueurs des différents articles sont très inégales. De plus, l'article le plus long comporte 230 000 caractères, soit certainement plusieurs dizaines de milliers de tokens après tokenization. Il n'est donc pas envisageable d'avoir une équivalence simple où un document correspond à un article.

De plus, la longueur totale du corpus étant très grande, son indexation risque de prendre beaucoup de temps. Nous allons donc travailler sur un sous-jeu de données très réduit (10 articles) à but d'exemple, puis nous utiliserons un index déjà pré-calculé pour avoir accès à plus de données.

La première étape va être de convertir les données existantes en documents, c'est-à-dire des données textuelles possédant des métadonnées supplémentaires. Ici, la métadonnée principale qui nous intéresse est la source, c'est-à-dire d'où le document est issu.

La conversion en documents se fait en deux temps :
1. Commencer par définir un gros document par article
2. Découper tous les articles en sous-documents de taille fixe.

In [ ]:
# Sélection de 10 articles seulement
ds = load_dataset("Sketched33/Cities_Wikipedia_Information", split="train[:10]")

# 1. Création des gros documents
source_docs = []
for doc in ds:
    source_docs.append(Document(
        page_content=doc["wikipedia_content"],
        metadata={"source": "Wikipedia - " + doc["city_name"]}
    ))

Avant de procéder à la sous-étape 2, il nous faut déjà réfléchir à la suite des opérations. En effet, on souhaite éviter de découper les documents au milieu des mots. Il nous faut donc choisir un tokeniseur.

L'étape suivante consistera à indexer les documents, ce que nous ferons grâce à un modèle qui convertira chaque sous-document en embeddings. Il faut donc que notre tokeniseur corresponde à ce modèle.

Plus précisément, la conversion en embeddings se fait grâce à un modèle de type *encodeur*. Nous utiliserons ici les modèles [GTE](https://arxiv.org/abs/2308.03281), et nous choisissons la version la plus petite (384 dimensions), qui est amplement suffisante pour ce projet.

Remarque : La tokenisation et la génération des embeddings sont ici effectuées en local, et non pas dans le nuage. Cependant, il est possible de réaliser ces étapes en utilisant l'API de Huggingface.

In [ ]:
# Création du tokeniseur
embedding_model_name = "thenlper/gte-small"
tokenizer = AutoTokenizer.from_pretrained(embedding_model_name)


# Création du découpeur de documents. Ce découpeur tente de préserver la hiérarchie du texte si possible,
# en évitant de découper au milieu d'un paragraphe par exemple.
text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer,
    chunk_size=500,
    chunk_overlap=50,
    add_start_index=True,
    strip_whitespace=True,
)


chunk_size=500: Divise le texte en morceaux de 500 caractères. C'est la longueur maximale d'un "chunk".

chunk_overlap=50: Les morceaux se chevauchent sur 50 caractères. Cela assure une meilleure fluidité et évite de couper les phrases ou les idées importantes en deux.

add_start_index=True: Enregistre la position de chaque morceau dans le texte original. Utile pour savoir d'où vient l'information.

strip_whitespace=True: Supprime les espaces inutiles au début et à la fin de chaque morceau, pour un traitement plus propre.

In [ ]:
# 2. Exécution du découpage, et création d'une liste de tous les sous-documents.
# On vérifie ici également qu'il n'y ait pas de doublons.
docs_processed = []
docs_added = set()
for doc in tqdm(source_docs):
    new_docs = text_splitter.split_documents([doc])
    for new_doc in new_docs:
        if new_doc.page_content not in docs_added:
            docs_added.add(new_doc.page_content)
            docs_processed.append(new_doc)

print()
print(f'Total : {len(docs_processed)} documents')

pd.DataFrame({**doc.metadata, 'page_content': doc.page_content} for doc in docs_processed)

100%|██████████| 10/10 [00:10<00:00,  1.03s/it]


Total : 592 documents


,source,start_index,page_content
0,Wikipedia - Tokyo,0,"Tokyo (; Japanese: 東京, Tōkyō, [toːkʲoː] ), off..."
1,Wikipedia - Tokyo,1271,"Prior to the 17th century, Tokyo was predomina..."
2,Wikipedia - Tokyo,-1,"Great Kantō earthquake, and the city was later..."
3,Wikipedia - Tokyo,2321,home to 29 of the world's largest 500 companie...
4,Wikipedia - Tokyo,-1,"Line, which opened in 1927, is the oldest unde..."
...,...,...,...
587,Wikipedia - Mexico City,78566,=== Media ===\nMexico City is Mexico's most im...
588,Wikipedia - Mexico City,79934,=== Sports ===\nAssociation football is the co...
589,Wikipedia - Mexico City,81793,The Autódromo Hermanos Rodríguez is the main v...
590,Wikipedia - Mexico City,83695,Other sports facilities in Mexico City are the...


On observe que malgré le fait de nous être restreints à 10 articles, nous avons déjà plusieurs centaines de documents.

## Indexation et classement

La prochaine étape consiste à indexer les documents. On rappelle que cette étape a deux buts :
- Affecter à chaque document un **index**, c'est-à-dure une donnée numérique que l'on peut comparer aux autres
- Permettre de stocker une grande quantité de données de manière **efficace** (il doit être facile de chercher dans les données, ainsi que de retrouver à quel document correspond un embedding).




In [ ]:
# Chargement du modèle encodeur
embedding_model = HuggingFaceEmbeddings(model_name=embedding_model_name)

<ipython-input-8-682958750292>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name=embedding_model_name)


#### Approche manuelle/naïve

Cette sous-section est donnée à titre pédagogique; nous verrons ensuite comment utiliser une bibliothèque pour effectuer tout cela à notre place.

Nous allons ici simplement créer une liste de paires `(embedding, document)`.

In [ ]:
# 3. Indexation (manuelle)
indexed_docs = [] # Création d'une liste vide pour stocker les documents indexés
for doc in tqdm(docs_processed): # Itération sur chaque document dans la liste 'docs_processed', avec une barre de progression
    indexed_docs.append((embedding_model.embed_documents([doc.page_content])[0], doc)) # On crée un tuple contenant l'embedding du document et le document lui-même, puis on l'ajoute à la liste 'indexed_docs'

100%|██████████| 592/592 [03:19<00:00,  2.97it/s]


Une fois cela fait, on peut classer les documents en mesurant leur distance par rapport à une requête donnée. La mesure de distance la plus courante est l'inverse de la similarité cosinus, qui évalue à quel point deux vecteurs d'embeddings sont alignés dans l'espace.

Un test sur une requête d'exemple, afin de voir les documents correspondant le mieux à cette requête :

In [ ]:
# 4. Classement (manuel)
query = "percentage of Christians in Mumbai"
query_embeddings = embedding_model.embed_documents([query])[0]  # Génération de l'embedding de la requête

# Définition d'une fonction pour calculer la distance entre deux embeddings
# La distance utilisée ici est l'inverse de la similarité cosinus
def embedding_distance(a, b):
    return 1 - np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# Calcul de la distance de la requête à chaque document
sorted_docs = []
for doc in indexed_docs:
    dist = embedding_distance(query_embeddings, doc[0]) # Calcul de la distance entre l'embedding de la requête et l'embedding du document
    sorted_docs.append((dist, doc[1]))

# Affichage des trois documents les plus proches
sorted_docs.sort(key=lambda doc: doc[0]) # Tri des documents par distance croissante (les documents les plus pertinents en premier)
for dist, text in sorted_docs[:5]:
    print(f"================ Distance : {dist} ================")
    print(f"{text.page_content}")
    print()
    print()
    print()

================ Distance : 0.07230691686026969 ================
=== Ethnic groups and religions ===

The religious groups represented in Greater Mumbai as of 2011 include Hindus (65.99%), Muslims (20.65%), Buddhists (4.85%), Jains (4.10%), Christians (3.27%) and Sikhs (0.49%). The linguistic/ethnic demographics in the Greater Mumbai Area are: Maharashtrians (32%), Gujaratis (20%), with the rest hailing from other parts of India.Native Christians include East Indian Catholics, who were converted by the Portuguese during the 16th century, while Goan and Mangalorean Catholics also constitute a significant portion of the Christian community of the city. Jews settled in Mumbai during the 18th century. The Bene Israeli Jewish community of Mumbai, who migrated from the Konkan villages, south of Mumbai, are believed to be the descendants of the Jews of Israel who were shipwrecked off the Konkan coast, probably in the year 175 BCE, during the reign of the Greek ruler, Antiochus IV Epiphanes. M

#### Approche pratique

En pratique, les documents indexés sont stockés dans des [**bases de données vectorielles**](https://en.wikipedia.org/wiki/Vector_database), aussi appelées *vector stores*. Celles-ci sont très efficace et permettent de stocker des quantités énormes de données. Lors de l'indexation, certains pré-calculs sont déjà effectués afin d'accélérer la partie recherche : en effet, il serait impraticable de devoir calculer la distance entre une requête et chacun des documents de la collection, qui peut en contenir des dizaines de millions.

Nous utiliserons ici la bibliothèque [FAISS](https://arxiv.org/abs/2401.08281).


In [ ]:
# Le bloc ci-dessous peut être entièrement remplacé par une seule ligne, la suivante :
#
# vectordb = FAISS.from_documents(docs_processed, embedding=embedding_model, distance_strategy='cosine') #Cette ligne de code crée un index de recherche FAISS
#à partir d'une liste de documents docs_processed en utilisant le modèle d'intégration embedding_model et la stratégie de distance cosinus pour la similarité sémantique.



#
# Cependant, par défaut, FAISS n'affiche pas l'avancement des calculs.
# Comme cette tâche prend quelques minutes, nous utilisons ici la bibliothèque tqdm
# pour afficher une barre de progression.

# 3. Indexation et classement (efficace)
vectordb = None # la variable vectordb à None, elle contiendra la base de données vectorielle après l'indexation.
with tqdm(total=len(docs_processed), desc="Création de l'index") as pbar:  # Utilisation de tqdm pour afficher une barre
    for d in docs_processed:
        if vectordb:
            vectordb.add_documents([d]) # Si la base vectordb existe déjà, le document actuel est ajouté à la base.
        else:
            # On crée la base de données en lui passant notre modèle d'embeddings ainsi que la métrique de classement voulu,
            # à savoir ici la distance cosinus
            vectordb = FAISS.from_documents([d], embedding=embedding_model, distance_strategy='cosine')
        pbar.update(1)


Création de l'index: 100%|██████████| 592/592 [03:36<00:00,  2.73it/s]


La recherche des documents les plus proches de notre requête ci-dessus peut désormais être effectuée quasi-instantanément :

In [ ]:
sorted_docs = vectordb.similarity_search(query, k=5)

for doc in sorted_docs:
    print('=' * 100)
    print(doc.page_content)

=== Ethnic groups and religions ===

The religious groups represented in Greater Mumbai as of 2011 include Hindus (65.99%), Muslims (20.65%), Buddhists (4.85%), Jains (4.10%), Christians (3.27%) and Sikhs (0.49%). The linguistic/ethnic demographics in the Greater Mumbai Area are: Maharashtrians (32%), Gujaratis (20%), with the rest hailing from other parts of India.Native Christians include East Indian Catholics, who were converted by the Portuguese during the 16th century, while Goan and Mangalorean Catholics also constitute a significant portion of the Christian community of the city. Jews settled in Mumbai during the 18th century. The Bene Israeli Jewish community of Mumbai, who migrated from the Konkan villages, south of Mumbai, are believed to be the descendants of the Jews of Israel who were shipwrecked off the Konkan coast, probably in the year 175 BCE, during the reign of the Greek ruler, Antiochus IV Epiphanes. Mumbai is also home to the largest population of Parsi Zoroastrian

Pour le reste du TP, on utilisera désormais une base de données pré-chargée; elle a été créée excatement de la même manière que celle que vous venez de réaliser, mais celle-ci indexe les sous-documents correspondant aux 3 000 premiers articles du jeu de données (contrairement à nos 10). De plus, elle utilise des paramètres de découpage différents (documents de 1024 caractères, avec 100 caractères de chevauchement), soit un total de 18 081 sous-documents.

In [ ]:
# Chargement de la base de données pré-enregistrée
!gdown 1bBDiCmbo2aL1ZL50T7bLc12CS94vc9R8
!mkdir faiss_index
!unzip index -d faiss_index
vectordb = FAISS.load_local("faiss_index", embedding_model, allow_dangerous_deserialization=True)

# Note : Une base de données peut également être enregistrée sur le disque, de manière suivante :
# vectordb.save_local("faiss_index")

#Ce code télécharge, extrait et charge un index FAISS pré-pré-chargéé pour permettre des recherches rapides dans une base de données vectorielle.

Downloading...
From (original): https://drive.google.com/uc?id=1bBDiCmbo2aL1ZL50T7bLc12CS94vc9R8
From (redirected): https://drive.google.com/uc?id=1bBDiCmbo2aL1ZL50T7bLc12CS94vc9R8&confirm=t&uuid=36b7cc88-2732-49f9-85e5-5db17622385a
To: /content/index.zip
100% 49.2M/49.2M [00:00<00:00, 256MB/s]
mkdir: cannot create directory ‘faiss_index’: File exists
Archive:  index.zip
replace faiss_index/index.pkl? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: faiss_index/index.pkl   
replace faiss_index/index.faiss? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: faiss_index/index.faiss  


## Génération

Pour la partie génération de texte, nous aurons besoin d'un LLM. Nous utiliserons ici Qwen2.5-72B-Instruct, un très gros modèle de 72 milliards de paramètres créé par Alibaba Cloud.

De plus, nous utiliserons la classe `HfApiEngine`, qui permet une utilisation encore plus simplifiée des LLM. Cette classe utilise en effet les paramètres de génération par défaut correspondant au LLM visé.


In [ ]:
# TODO : Remplacer la valeur ci-dessous par votre token d'API.
# Si vous n'y avès plus accès, vous pouvez le régénérer ou en créer un nouveau
# sur la page correspondante (https://huggingface.co/settings/tokens)
api_token = "hf_"

# Note : L'utilisation d'un token d'API gratuit ne permet normalement pas
# d'accèder à des modèles de plus de 10 milliards de paramètres.
# Si la ligne ci-dessous ne fonctionne pas, modifiez le nom du modèle
# en utilisant par exemple "meta-llama/Meta-Llama-3-8B-Instruct" comme au TP précédent.
llm_engine = HfApiEngine(model="Qwen/Qwen2.5-72B-Instruct", token=api_token)

messages = [
    {"role": "user", "content": "Hello, how are you?"},
]

llm_engine(messages)

"Hello! I'm doing well, thank you for asking. How about you? How can I assist you today?"

Nous sommes désormais en possession de la requête utilisateur (`query`) ainsi que des trois documents les plus proches (`sorted_docs`). Il ne nous reste plus qu'à passer tous ces éléments au LLM afin qu'il génère une réponse pertinente.

In [ ]:
sorted_docs[0]

Document(metadata={'source': 'Wikipedia - Mumbai', 'start_index': 48013}, page_content='=== Ethnic groups and religions ===\n\nThe religious groups represented in Greater Mumbai as of 2011 include Hindus (65.99%), Muslims (20.65%), Buddhists (4.85%), Jains (4.10%), Christians (3.27%) and Sikhs (0.49%). The linguistic/ethnic demographics in the Greater Mumbai Area are: Maharashtrians (32%), Gujaratis (20%), with the rest hailing from other parts of India.Native Christians include East Indian Catholics, who were converted by the Portuguese during the 16th century, while Goan and Mangalorean Catholics also constitute a significant portion of the Christian community of the city. Jews settled in Mumbai during the 18th century. The Bene Israeli Jewish community of Mumbai, who migrated from the Konkan villages, south of Mumbai, are believed to be the descendants of the Jews of Israel who were shipwrecked off the Konkan coast, probably in the year 175 BCE, during the reign of the Greek ruler, 

In [ ]:
# 4. Génération

# TODO : Écrivez un prompt indiquant au LLM qu'il doit répondre à la requête de l'utilisateur,
# en pouvant éventuellement s'aider des cinq documents de contexte.
# L'exécution de cette cellule de code doit produire une réponse textuelle contenant la solution à la requête.
#
# Vous pouvez utiliser les variables de formatage suivantes :
# {query} - sera remplacé par la requête de l'utilisateur
# {doc1}, {doc2}, ..., {doc5} - sera remplacé par le texte des cinq documents les plus proches de la requête
# {src1}, {src2}, ..., {src5} - sera remplacé par la source des cinq documents les plus proches de la requête
sorted_docs = vectordb.similarity_search(query, k=5)
query = "C'est quoi la population de Paris?"
prompt = """Répondre à la question de l'utilisateur en utilisant le contexte fourni. \
Si la réponse ne se trouve pas dans le contexte, répondre par "Je suis désolé, \
je n'ai pas assez d'informations pour répondre à votre question."

**Question de l'utilisateur :** {query}

**Contexte disponible :**

{doc1} (Source : {src1})
{doc2} (Source : {src2})
{doc3} (Source : {src3})
{doc4} (Source : {src4})
{doc5} (Source : {src5})
"""

doc1, doc2, doc3, doc4, doc5 = [doc.page_content for doc in sorted_docs[:5]]
src1, src2, src3, src4, src5 = [doc.metadata['source'] for doc in sorted_docs[:5]]
messages = [
    {"role": "user", "content": prompt.format(query=query,
                                              doc1=doc1, doc2=doc2, doc3=doc3, doc4=doc4, doc5=doc5,
                                              src1=src1, src2=src2, src3=src3, src4=src4, src5=src5)},
]

print(llm_engine(messages))

La population officielle estimée de Paris au 1er janvier 2023 était de 2,102,650 habitants, selon l'INSEE, l'agence statistique officielle de la France.


In [ ]:
sorted_docs

[Document(metadata={'source': 'Wikipedia - Paris', 'start_index': 38070}, page_content='== Demographics ==\n\nThe official estimated population of the City of Paris on 1 January 1, 2023 was 2,102,650, down from 2,165,423 on 1 January 2022, according to the INSEE, the official French statistical agency. According to INSEE, the population has dropped by 122,919, or about five percent, over the past decade. The Mayor of Paris, Anne Hidalgo, declared that this illustrated the "de-densification" of the city, creating more green space and less crowding. Despite the drop, Paris remains the most densely-populated city in Europe, with 252 residents per hectare, not counting parks. This drop was attributed partly to a lower birth rate, the departure of middle-class residents and the possible loss of housing in the city due to short-term rentals for tourism.Paris is the fourth largest municipality in the European Union, following Berlin, Madrid and Rome. Eurostat places Paris (6.5 million people)

In [ ]:
# 4. Génération

# TODO : Écrivez un prompt indiquant au LLM qu'il doit répondre à la requête de l'utilisateur,
# en pouvant éventuellement s'aider des cinq documents de contexte.
# L'exécution de cette cellule de code doit produire une réponse textuelle contenant la solution à la requête.
#
# Vous pouvez utiliser les variables de formatage suivantes :
# {query} - sera remplacé par la requête de l'utilisateur
# {doc1}, {doc2}, ..., {doc5} - sera remplacé par le texte des cinq documents les plus proches de la requête
# {src1}, {src2}, ..., {src5} - sera remplacé par la source des cinq documents les plus proches de la requête

query = "What is the average elevation in Jakarta and what is the highest temperature ever recorded in Tokyo?"
prompt = """Answer the user's question using the provided context. Try to identify the most relevant context for answering the question and cite it using bracketed numbers like this: [1]. If the information is spread across multiple sources, combine them to create a comprehensive answer and cite all relevant sources accordingly.

If the provided context is insufficient to answer the question, state that you are unable to provide an answer and explain why. Avoid using phrases like "I do not have enough information" or "My internal knowledge base does not contain." Instead, provide specific reasons why the context is insufficient, such as "The provided context primarily focuses on [topic] and does not contain information about [user's question]."

**Question:** {query}

**Context:**

{doc1} (Source : {src1})
{doc2} (Source : {src2})
{doc3} (Source : {src3})
{doc4} (Source : {src4})
{doc5} (Source : {src5})

**SOURCES:**
[1] {src1}
[2] {src2}
[3] {src3}
[4] {src4}
[5] {src5}
"""

sorted_docs = vectordb.similarity_search(query, k=5)
doc1, doc2, doc3, doc4, doc5 = [doc.page_content for doc in sorted_docs[:5]]
src1, src2, src3, src4, src5 = [doc.metadata['source'] for doc in sorted_docs[:5]]
messages = [
    {"role": "user", "content": prompt.format(query=query,
                                              doc1=doc1, doc2=doc2, doc3=doc3, doc4=doc4, doc5=doc5,
                                              src1=src1, src2=src2, src3=src3, src4=src4, src5=src5)},
]

print(llm_engine(messages))

The provided context does not contain specific information about the average elevation in Jakarta or the highest temperature ever recorded in Tokyo. The context for Jakarta primarily discusses the city's climate, population, and some geographical features, but it does not mention the average elevation [1]. For Tokyo, the context provides information about the climate, including the record high temperature of 39.5 °C (103.1 °F) on July 20, 2004 [2], but it does not provide details on the city's elevation.

To answer the specific parts of your question:
- The average elevation in Jakarta is not provided in the context. However, general sources often report Jakarta's average elevation to be around 7 meters (23 feet) above sea level, with some parts of the city being below sea level.
- The highest temperature ever recorded in Tokyo is 39.5 °C (103.1 °F), which was recorded on July 20, 2004 [2].

If you need more precise or detailed information on Jakarta's elevation, you may need to consul

In [ ]:
# 4. Génération

# TODO : Écrivez un prompt indiquant au LLM qu'il doit répondre à la requête de l'utilisateur,
# en pouvant éventuellement s'aider des cinq documents de contexte.
# L'exécution de cette cellule de code doit produire une réponse textuelle contenant la solution à la requête.
#
# Vous pouvez utiliser les variables de formatage suivantes :
# {query} - sera remplacé par la requête de l'utilisateur
# {doc1}, {doc2}, ..., {doc5} - sera remplacé par le texte des cinq documents les plus proches de la requête
# {src1}, {src2}, ..., {src5} - sera remplacé par la source des cinq documents les plus proches de la requête

query = "What cities have the most tourism?"
prompt = """Answer the user's question using the provided context. Try to identify the most relevant context for answering the question and cite it using bracketed numbers like this: [1]. If the information is spread across multiple sources, combine them to create a comprehensive answer and cite all relevant sources accordingly.

If the provided context is insufficient to answer the question, state that you are unable to provide an answer and explain why. Avoid using phrases like "I do not have enough information" or "My internal knowledge base does not contain." Instead, provide specific reasons why the context is insufficient, such as "The provided context primarily focuses on [topic] and does not contain information about [user's question]."

**Question:** {query}

**Context:**

{doc1} (Source : {src1})
{doc2} (Source : {src2})
{doc3} (Source : {src3})
{doc4} (Source : {src4})
{doc5} (Source : {src5})

**SOURCES:**
[1] {src1}
[2] {src2}
[3] {src3}
[4] {src4}
[5] {src5}
"""
sorted_docs = vectordb.similarity_search(query, k=5)
doc1, doc2, doc3, doc4, doc5 = [doc.page_content for doc in sorted_docs[:5]]
src1, src2, src3, src4, src5 = [doc.metadata['source'] for doc in sorted_docs[:5]]
messages = [
    {"role": "user", "content": prompt.format(query=query,
                                              doc1=doc1, doc2=doc2, doc3=doc3, doc4=doc4, doc5=doc5,
                                              src1=src1, src2=src2, src3=src3, src4=src4, src5=src5)},
]

print(llm_engine(messages))

Based on the provided context, here are some cities with significant tourism:

1. **Venice, Italy**: Venice is a major tourist destination, receiving between 22 million to 30 million visitors annually. It is known for its art, architecture, and cultural heritage, including attractions like St Mark's Basilica, the Doge's Palace, and the Grand Canal [2].

2. **Milan, Italy**: Milan is the world's 15th-most visited city, with 8.81 million registered international arrivals in 2018. It is popular for its historical and cultural attractions, such as the Milan Cathedral, the Teatro alla Scala, and the Galleria Vittorio Emanuele II [3].

3. **Florence, Italy**: Florence is the world's 36th most visited city, with over 4.95 million arrivals in 2015. It is renowned for its rich cultural and historical sites, including the Uffizi and Accademia museums, and the Ponte Vecchio [4].

4. **Melbourne, Australia**: Melbourne is the second most visited city in Australia and the seventy-third most visited

In [ ]:
# 4. Génération

# TODO : Écrivez un prompt indiquant au LLM qu'il doit répondre à la requête de l'utilisateur,
# en pouvant éventuellement s'aider des cinq documents de contexte.
# L'exécution de cette cellule de code doit produire une réponse textuelle contenant la solution à la requête.
#
# Vous pouvez utiliser les variables de formatage suivantes :
# {query} - sera remplacé par la requête de l'utilisateur
# {doc1}, {doc2}, ..., {doc5} - sera remplacé par le texte des cinq documents les plus proches de la requête
# {src1}, {src2}, ..., {src5} - sera remplacé par la source des cinq documents les plus proches de la requête

query = "What cities have the most tourism in France?"
prompt = """Answer the user's question using the provided context. Try to identify the most relevant context for answering the question and cite it using bracketed numbers like this: [1]. If the information is spread across multiple sources, combine them to create a comprehensive answer and cite all relevant sources accordingly.

If the provided context is insufficient to answer the question, state that you are unable to provide an answer and explain why. Avoid using phrases like "I do not have enough information" or "My internal knowledge base does not contain." Instead, provide specific reasons why the context is insufficient, such as "The provided context primarily focuses on [topic] and does not contain information about [user's question]."

**Question:** {query}

**Context:**

{doc1} (Source : {src1})
{doc2} (Source : {src2})
{doc3} (Source : {src3})
{doc4} (Source : {src4})
{doc5} (Source : {src5})

**SOURCES:**
[1] {src1}
[2] {src2}
[3] {src3}
[4] {src4}
[5] {src5}
"""
sorted_docs = vectordb.similarity_search(query, k=5)
doc1, doc2, doc3, doc4, doc5 = [doc.page_content for doc in sorted_docs[:5]]
src1, src2, src3, src4, src5 = [doc.metadata['source'] for doc in sorted_docs[:5]]
messages = [
    {"role": "user", "content": prompt.format(query=query,
                                              doc1=doc1, doc2=doc2, doc3=doc3, doc4=doc4, doc5=doc5,
                                              src1=src1, src2=src2, src3=src3, src4=src4, src5=src5)},
]

print(llm_engine(messages))

The cities with the most tourism in France, based on the provided context, are Paris, Marseille, and Rennes. However, Paris stands out significantly as the primary tourist destination:

1. **Paris** [1]: Paris is not only the capital and most populous city of France but also one of the world's major centers of finance, diplomacy, commerce, culture, fashion, and gastronomy. It is renowned for its museums, architectural landmarks, and cultural events. Key attractions include the Louvre, Musée d'Orsay, and the historic district along the Seine, which is a UNESCO World Heritage Site. The city's major museums, such as the Louvre, attract millions of visitors annually, with the Louvre alone receiving 8.9 million visitors in 2023. Paris also hosts significant events like the French Open, the Tour de France, and the Olympic Games.

2. **Marseille** [4]: Marseille is the third-largest metropolitan area in France and is listed as a major center of art and history. It has a rich array of museums 

## Système complet

Il ne nous reste désormais plus qu'à combiner les différentes parties précédentes afin de générer un système complet.

In [ ]:
user_query = input('Assistant: Hello! How may I help you today?\n\nUser: ')
messages = []

while user_query != "Exit":
    sorted_docs = vectordb.similarity_search(user_query, k=5)
    doc1, doc2, doc3, doc4, doc5 = [doc.page_content for doc in sorted_docs[:5]]
    src1, src2, src3, src4, src5 = [doc.metadata['source'] for doc in sorted_docs[:5]]
    messages.append({"role": "user", "content": prompt.format(query=user_query,
                                                              doc1=doc1, doc2=doc2, doc3=doc3, doc4=doc4, doc5=doc5,
                                                              src1=src1, src2=src2, src3=src3, src4=src4, src5=src5)})
    answer = llm_engine(messages)
    print()
    print(f'Assistant: {answer}')
    print()
    messages.append({"role": "assistant", "content": answer})
    user_query = input('User: ')


messages = [{"role": "user", "content": initial_message}]

Assistant: Hello! How may I help you today?

User: What cities have the most tourism in Italy?

Assistant: Based on the provided context, the cities with the most tourism in Italy are:

1. **Rome** - Although not detailed in the provided context, it is mentioned that Milan ranks second in Italy for tourism, implying that Rome is the top destination. [1]
2. **Milan** - Known for its significant tourism, Milan attracted 8.81 million registered international arrivals in 2018, making it the second most visited city in Italy and the 15th most visited in the world. [1]
3. **Venice** - An important tourist destination, Venice sees up to 60,000 tourists per day and hosts between 22 to 30 million annual visitors. It is known for its art, architecture, and rich cultural heritage. [2]
4. **Florence** - A major cultural and historical destination, Florence had 4.95 million arrivals in 2015 and is the 36th most visited city in the world. It is renowned for its museums, art, and cultural attractions

KeyboardInterrupt: Interrupted by user

### Personnalisation

Qwen-2.5-72B est suffisamment grand pour avoir des facultés élevées, il est donc possible de lui donner un prompt complexe.

Ajustez votre prompt précédent afin d'améliorer l'expérience utilisateur. Voici une liste (non exhaustive) de fonctionnalités que vous pouvez rajouter :
- Liste des sources utilisés après la réponse, seulement si des sources ont été utilisées
- Numérotation des sources dans le texte de réponse et dans la liste des sources
- Lien vers la page Wikipedia
- Lien vers la section de la page Wikipedia, si applicable
- "Transparence" de l'interface de RAG : le LLM n'utilise jamais les mots "documents", "information" ou "context", mais fait référence aux données qu'il connait et aux sources externes

Un exemple de conversation possible est donné ci-dessous :

```
Assistant: Hello! How may I help you today?

User: What is the average elevation in Jakarta and what is the highest temperature ever recorded in Tokyo?

Assistant: Jakarta lies on the northwest coast of Java, and I do not know its specific average elevation, Jakarta is prone to flooding due to its low-lying areas near the coast, which suggests that much of the city is close to sea level or slightly above it [1].

The highest temperature ever recorded in Tokyo was 39.5 °C (103.1 °F) on July 20, 2004 [2].

 SOURCES:
 1. [Jakarta - Wikipedia](https://en.wikipedia.org/wiki/Jakarta#Geography)
 2. [Tokyo - Wikipedia](https://en.wikipedia.org/wiki/Tokyo#Climate)

User: What is the population of Grenoble?

Assistant: The population of the commune of Grenoble was 158,198 as of 2019 [1].

 SOURCES:
 1. [Grenoble - Wikipedia](https://en.wikipedia.org/wiki/Grenoble#Population)

User: What is 2+2?

Assistant: 2+2 equals 4.

User: What cities have the most tourism?

Assistant: Here are some cities known for having significant tourism:

1. **Venice** [1]: Venice attracts up to 60,000 tourists per day and estimates of the annual number of tourists range from 22 million to 30 million. The city is known for its art, architecture, and cultural heritage, making it a major tourist destination.

2. **Milan** [2]: In 2018, Milan recorded 8.81 million registered international arrivals, ranking it as the world's 15th-most visited city. It is a major tourist destination, with popular attractions like the Milan Cathedral, the Teatro alla Scala, and the Galleria Vittorio Emanuele II.

3. **Florence** [3]: Florence is a significant tourist destination, with the Uffizi selling over 1.93 million tickets in 2014. In 2016, the city had 20,588 hotel rooms in 570 facilities, with international visitors using 75% of the rooms. In 2014, the city had 8.5 million overnight stays and was ranked as the world's 36th most visited city with over 4.95 million arrivals.

4. **Melbourne** [4]: Melbourne is the second most visited city in Australia and the 73rd most visited city in the world. In 2018, it received 10.8 million domestic overnight tourists and 2.9 million international overnight tourists. Popular attractions include Federation Square, Queen Victoria Market, Crown Casino, and the National Gallery of Victoria.

 SOURCES:
 1. [Venice - Wikipedia](https://en.wikipedia.org/wiki/Venice#Tourism)
 2. [Milan - Wikipedia](https://en.wikipedia.org/wiki/Milan#Tourism)
 3. [Florence - Wikipedia](https://en.wikipedia.org/wiki/Florence#Tourism)
 4. [Melbourne - Wikipedia](https://en.wikipedia.org/wiki/Melbourne#Tourism)
```


## Agents haut niveau

La bibliothèque `transformers` offre une interface permettant d'abstraire plusieurs concepts vus précédemment. En effet, elle dispose d'une classe `Agent`, représentant un LLM capable d'utiliser des outils (classe `Tool`) pour mener à bien une tâche.

Commençons par définir un outil correspondant au fait de chercher un document dans notre base de données :

In [ ]:
class RetrieverTool(Tool):
    # Métadonnées décrivant l'outil, ainsi que le type d'entrées et de sorties qu'il accepte
    name = "retriever"
    description = "Retrieves documents using semantic similarity."
    inputs = {
        "query": {
            "type": "string",
            "description": "A query string to retrieve documents."
        }
    }
    output_type = "string"

    # Création de l'outil
    def __init__(self, vectordb):
        super().__init__()
        self.vectordb = vectordb

    # Appel de l'outil à partir d'une requête; ici, on veut renvoyer les documents les plus proches de la requête.
    def forward(self, query: str) -> str:
        if not isinstance(query, str):
            raise TypeError("Query must be a string.")
        docs = self.vectordb.similarity_search(query, k=5)
        return "\n".join([doc.page_content for doc in docs])


retriever_tool = RetrieverTool(vectordb=vectordb)

Nous sommes ici intéressés par l'utilisation de [ReAct](https://www.promptingguide.ai/techniques/react), une technique de prompting qui combine des étapes de planification (proches du promptage par chaîne de pensée) et des étapes d'action (utilisation d'outils).

In [ ]:
agent = ReactJsonAgent(
    tools=[retriever_tool],  # Liste des outils utilisables par l'agent
    llm_engine=llm_engine,
    max_iterations=5,  # Nombres d'itérations maximum
    verbose=False,
)

logging.getLogger().setLevel(logging.WARNING)  # Rend la sortie moins verbeuse

In [ ]:
user_question = "What are the names of the two main regions of Mumbai?"
response = agent.run(user_question)
print(f"Agent response: {response}")

======== New task ========
What are the names of the two main regions of Mumbai?
=== Agent thoughts:
Thought: I need to search for information about the main regions of Mumbai.
>>> Calling tool: 'retriever' with arguments: {'query': 'main regions of Mumbai'}
=== Agent thoughts:
Thought: I have the information I need to answer the question.
>>> Calling tool: 'final_answer' with arguments: {'answer': 'Mumbai City district and Mumbai Suburban district'}


Agent response: Mumbai City district and Mumbai Suburban district


## Questions et bonus

- Nous avons choisi arbitrairement de toujours renvoyer 5 documents par requête. Dans quelle(s) situation(s) cela peut-il poser problème et pourquoi ? Quelles idées potentielles auriez-vous pour remédier à cela ?

Problèmes:

Pertinence insuffisante: Si la requête est complexe ou ambigüe, 5 documents peuvent ne pas suffire pour fournir un contexte complet au LLM, ce qui peut mener à des réponses incomplètes ou inexactes.

Redondance: Si la requête est simple ou si les documents de la base de données sont très similaires, renvoyer 5 documents peut amener à de la redondance, gaspillant des ressources de calcul et rendant la réponse plus difficile à lire.

Biais potentiel: Le choix arbitraire de 5 documents peut introduire un biais dans les réponses du LLM, si les documents sélectionnés ne représentent pas l'ensemble des informations pertinentes sur le sujet.

Idées de remèdes:

Nombre de documents dynamique: Ajuster le nombre de documents renvoyés en fonction de la complexité de la requête et de la diversité des documents pertinents dans la base de données. On pourrait par exemple utiliser un seuil de similarité pour ne renvoyer que les documents significativement différents.

Clustering des documents: Regrouper les documents similaires en clusters et renvoyer un représentant de chaque cluster, pour éviter la redondance et augmenter la diversité des informations.

Métriques de pertinence: Utiliser des métriques plus avancées que la simple similarité cosinus pour sélectionner les documents les plus pertinents, par exemple en combinant des scores de similarité avec des informations sur la structure du document ou la fréquence des termes.

- Similairement, une seule requête est faite par message de l'utilisateur. Est-ce toujours approprié ? Quelles pistes d'amélioration pouvez-vous proposer ?


--


S'il vous reste du temps, voici deux tâches supplémentaires :

### Récupération

Bien que la distance entre embeddings soit la méthode la plus courante pour la récupération des documents, d'autres existent. LangChain en supporte [beaucoup](https://python.langchain.com/docs/integrations/retrievers/) d'autres. Certaines méthodes permettent même de faire directement des recherches en ligne, et donc de ne pas avoir de collection locale : c'est le cas par exemple du récupérateur PubMed (qui cherche des articles de recherche en médecine) ou de celui de Google Drive (qui cherche des documents dans un dossier Google Drive).

1. En utilisant les pages de documentation correspondantes, modifiez le code du TP pour plutôt utiliser [la TF-IDF](https://python.langchain.com/docs/integrations/retrievers/tf_idf/). Observez-vous une différence significative en termes de performance ?

2. LangChain dispose déjà d'une méthode de récupération [basée sur Wikipedia](https://python.langchain.com/docs/integrations/retrievers/wikipedia/). Modifiez le code du TP pour utiliser celle-ci plutôt que notre base de données réduite.



In [ ]:
!pip install scikit-learn
from langchain.vectorstores import SKLearnVectorStore
from langchain.text_splitter import CharacterTextSplitter
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# Split documents and create a list of texts and metadata
texts = []
metadatas = []
for doc in source_docs:
    splits = text_splitter.split_text(doc.page_content)
    texts.extend(splits)
    metadatas.extend([doc.metadata] * len(splits))  # Associate metadata with splits

# Create and fit the vector store
vectordb = SKLearnVectorStore.from_texts(texts, vectorizer, metadatas=metadatas)